### Intial setup

Installation of libraries

In [1]:
!pip install -U autogen-agentchat autogen-ext[openai]
!pip install torch transformers sentencepiece

  Using cached jsonref-1.1.0-py3-none-any.whl.metadata (2.7 kB)
     ---------------------------------------- 0.0/66.6 kB ? eta -:--:--
     ---------------------------------------- 66.6/66.6 kB 1.8 MB/s eta 0:00:00
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached typing_inspection-0.4.0-py3-none-any.whl.metadata (2.6 kB)
   ---------------------------------------- 0.0/105.5 kB ? eta -:--:--
   ---------------------------------------- 105.5/105.5 kB 3.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/94.7 kB ? eta -:--:--
   ---------------------------------------- 94.7/94.7 kB 5.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/662.0 kB ? eta -:--:--
   ------------------- -------------------- 317.4/662.0 kB 6.5 MB/s eta 0:00:01
   ---------------------------------------- 662.0/662.0 kB 6.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/893.9 kB ? eta -:--:--
   ---------------------------- --

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-cloud-auth 0.1.3 requires pydantic<2.0, but you have pydantic 2.11.4 which is incompatible.
googleapis-common-protos 1.63.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.29.4 which is incompatible.
google-api-core 2.18.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.29.4 which is incompatible.
langchain 0.3.3 requires langchain-core<0.4.0,>=0.3.10, but you have langchain-core 0.1.43 which is incompatible.
langchain-community 0.3.2 requires langchain-core<0.4.0,>=0.3.10, but you have langchain-core 0.1.43 which is incompatible.
langchain-community 0.3.2 requires langsmith<0.2.0,>=0.1.125, but you have langsmith 0.1.48 which is incompatible.

   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   - ------------------------------------- 41.0/991.5 kB 991.0 kB/s eta 0:00:01
   --------------------- ------------------ 532.5/991.5 kB 8.4 MB/s eta 0:00:01
   ---------------------------------------- 991.5/991.5 kB 9.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


API key setup

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

### Multi Agent Setup

In [17]:
from autogen.agentchat import AssistantAgent

#### Fine tuned Agents

wrapper agent for autogen

In [28]:
from autogen.agentchat import AssistantAgent

class LocalModelWrapperAgent(AssistantAgent):
    def __init__(self, name: str, model_agent):
        super().__init__(name=name)
        self.model_agent = model_agent

    def generate_reply(self, messages, sender=None, **kwargs):
        # Safe wrapper with fallback
        try:
            raw_json = messages[0]["content"]  # Expecting first message to be UI JSON
            reply = self.model_agent.handle(raw_json)
            return reply
        except Exception as e:
            return f"[{self.name}] Error: {str(e)}"


Semantic Agent

In [19]:
from agents.semantic_agent.agent import SemanticAgent

semantic_model = SemanticAgent(model_dir="trusha88/t5-semantic-agent")
semantic_agent = LocalModelWrapperAgent(name="semantic-agent", model_agent=semantic_model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Contrast Agent

In [20]:
from agents.contrast_agent.agent import ContrastAgent

contrast_model = ContrastAgent(model_dir="virajns2/contrast-violation-t5")
contrast_agent = LocalModelWrapperAgent(name="contrast-agent", model_agent=contrast_model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### GPT based Agents

In [21]:
# Visually Impaired Agent
visually_impaired_agent = AssistantAgent(
    name="VisuallyImpairedAgent",
    system_message="Simulate a user navigating via screen reader. Identify missing labels, improper heading structures, and ARIA attribute issues.",
    llm_config={"model": "gpt-4", "temperature": 0}
)

In [22]:
# Motor-Impaired Agent
motor_impaired_agent = AssistantAgent(
    name="MotorImpairedAgent",
    system_message="Simulate keyboard-only navigation. Detect elements not accessible via keyboard and lack of focus indicators.",
    llm_config={"model": "gpt-4", "temperature": 0}
)


In [23]:
# Color-Blind Agent
color_blind_agent = AssistantAgent(
    name="ColorBlindAgent",
    system_message="Analyze color usage for accessibility. Identify issues with color contrast and reliance on color cues.",
    llm_config={"model": "gpt-4", "temperature": 0}
)

In [24]:
fixing_agent = AssistantAgent(
    name="FixingAgent",
    system_message="Provide code-level fixes for identified accessibility issues based on best practices and guidelines, based on the problems stated by everyone",
    llm_config={"model": "gpt-4", "temperature": 0}
)

Configure Agent Interactions

In [25]:
from autogen.agentchat import UserProxyAgent, GroupChat, GroupChatManager

user_proxy = UserProxyAgent(
    name="UserProxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    code_execution_config={"use_docker": False}
)

group_chat = GroupChat(
    agents=[user_proxy, semantic_agent, contrast_agent, visually_impaired_agent, motor_impaired_agent, color_blind_agent, fixing_agent],
    messages=[],
    max_round=5
)

manager = GroupChatManager(groupchat=group_chat, llm_config={"model": "gpt-4"})

Run the Evaluation

In [34]:
#TODO: FETCH RESPECTIVE DATA FROM TEST SET AND PASS TO SYSTEM
with open("test_data/test.json", "r", encoding="utf-8") as f:
    ui = f.read()
ui = "role: link, fg: 120,120,120, bg: 255,255,255, contrast: 2.9"



Feed back loop

In [35]:
contrast_agent.model_agent.handle(ui) 

'The link element has a foreground color of RGB(120,120,120) and background color of RGB(255,255,255) resulting in a contrast ratio of 2.9. This is below the WCAG recommended minimum of 4.5:1 for normal text.'

In [36]:
from autogen.agentchat import UserProxyAgent, GroupChat, GroupChatManager

# Step 1: Define agents (you already did this)
user_proxy = UserProxyAgent(
    name="UserProxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    code_execution_config={"use_docker": False}
)

# Step 2: Set up group chat (we'll use this for message context)
group_chat = GroupChat(
    agents=[user_proxy, semantic_agent, contrast_agent, visually_impaired_agent, motor_impaired_agent, color_blind_agent, fixing_agent],
    messages=[],
    max_round=5
)
manager = GroupChatManager(groupchat=group_chat, llm_config={"model": "gpt-4"})

# Step 3: Load JSON
with open("test_data/test.json", "r", encoding="utf-8") as f:
    ui = f.read()

    
ui = "role: link, fg: 120,120,120, bg: 255,255,255, contrast: 2.9"

In [37]:
# Step 1: Load test input
# with open("test_data/test.json", "r", encoding="utf-8") as f:
#     ui = f.read()

# Step 2: Reset group chat with initial user input
group_chat.messages = [{"role": "user", "content": ui}]

# Step 3: Manually invoke each accessibility agent
ordered_agents = [
    # semantic_agent,  # include this if you want
    contrast_agent,
    visually_impaired_agent,
    motor_impaired_agent,
    color_blind_agent,
]

for agent in ordered_agents:
    try:
        reply = agent.generate_reply(group_chat.messages)

        # ✅ Append with valid OpenAI role and embed agent name in content
        group_chat.messages.append({
            "role": "assistant", 
            "content": f"[{agent.name}]\n{reply}"
        })

        print(f"[{agent.name}]: {reply}\n")

    except Exception as e:
        print(f"❌ Error in {agent.name}: {e}")

# Step 4: Fixing agent responds last
try:
    final_reply = fixing_agent.generate_reply(group_chat.messages)
    group_chat.messages.append({
        "role": "assistant",
        "content": f"[{fixing_agent.name}]\n{final_reply}"
    })
    print(f"[{fixing_agent.name}]: {final_reply}")
except Exception as e:
    print(f"❌ Error in FixingAgent: {e}")


[contrast-agent]: The link element has a foreground color of RGB(120,120,120) and background color of RGB(255,255,255) resulting in a contrast ratio of 2.9. This is below the WCAG recommended minimum of 4.5:1 for normal text.

[VisuallyImpairedAgent]: [screen-reader]
"Unlabeled link."

[accessibility-issue]
The link element is missing a descriptive label or text. This makes it difficult for screen reader users to understand the purpose of the link.

[MotorImpairedAgent]: [KeyboardNavigationAgent]
[tab]
"Link."

[accessibility-issue]
The link element is accessible via keyboard navigation. However, there is no visible focus indicator when the link is selected. This makes it difficult for keyboard-only users to track their navigation.

[ColorBlindAgent]: [ColorBlindAgent]
[ColorContrastAgent]
The link element has a foreground color of RGB(120,120,120) and a background color of RGB(255,255,255). This results in a contrast ratio of 2.9, which is below the minimum recommended contrast ratio 